## Utility Functions
Core helper functions for sparse Pauli manipulation and commutation testing.

In [45]:
# One-liner sparse commutation helper (Stim backend) that accepts sparse strings or Stim PauliStrings.
import importlib.util, pathlib
from stim_import import stim

def _to_sparse_spec(p):
    if isinstance(p, str):
        return p
    s = str(p)
    if s and s[0] in '+-':
        s = s[1:]
    tokens = []
    for idx, ch in enumerate(s):
        if ch in ('X', 'Y', 'Z'):
            tokens.append(f"{ch}{idx}")
    return ' '.join(tokens)

def commutes_sparse(pauli_a, paulis_b):
    """Return [a commutes with b_i] using sparse tokens like "Z0 Z2 X3".

    paulis_b can be an iterable of sparse strings or Stim PauliStrings (or a single one).
    Length is unified across inputs (max qubit index + 1)."""
    here = pathlib.Path('.').resolve()
    ph_path = here / 'pauli_handling.py'
    spec = importlib.util.spec_from_file_location('pauli_handling', ph_path)
    pauli_handling = importlib.util.module_from_spec(spec)
    assert spec and spec.loader
    spec.loader.exec_module(pauli_handling)
    parse_sparse_pauli = pauli_handling.parse_sparse_pauli

    if isinstance(paulis_b, (str, stim.PauliString)):
        paulis_list = [_to_sparse_spec(paulis_b)]
    else:
        paulis_list = [_to_sparse_spec(p) for p in paulis_b]

    specs = [_to_sparse_spec(pauli_a)] + paulis_list
    max_idx = -1
    for s in specs:
        for tok in s.replace('*', ' ').split():
            tok = tok.strip()
            if not tok:
                continue
            if tok[0].upper() in 'XYZ' and tok[1:].isdigit():
                max_idx = max(max_idx, int(tok[1:]))
    n = max_idx + 1 if max_idx >= 0 else 0
    a_ps = parse_sparse_pauli(specs[0], num_qubits=n)
    b_ps = [parse_sparse_pauli(t, num_qubits=n) for t in paulis_list]
    return [a_ps.commutes(b) for b in b_ps]

def ps_to_sparse(p: stim.PauliString) -> str:
    """Convert Stim PauliString to sparse representation."""
    s = str(p)
    if s and s[0] in '+-':
        s = s[1:]
    tokens = []
    for idx, ch in enumerate(s):
        if ch in ('X', 'Y', 'Z'):
            tokens.append(f"{ch}{idx}")
    return ' '.join(tokens)


In [46]:
# Import required modules for Clifford synthesis
import sys, pathlib
from stim_import import stim

# Ensure backend is in path
here = pathlib.Path('.').resolve()
if str(here) not in sys.path:
    sys.path.insert(0, str(here))

from clifford_from_map import synthesize_clifford_from_sd_pairs
from pauli_handling import parse_sparse_pauli, canonicalize_sparse_pauli
from stabilizer_canonicalization import canonicalize_stabilizer_code


## Test Functions
Helper functions to validate Clifford synthesis results.

In [47]:
def test_clifford_synthesis(pairs, num_qubits, description="Test"):
    """Test Clifford synthesis from stabilizer-destabilizer pairs."""
    print(f"\n=== {description} ===")
    print(f"Input pairs (S, D): {pairs}")
    print(f"Number of qubits: {num_qubits}")
    
    try:
        res = synthesize_clifford_from_sd_pairs(pairs, num_qubits=num_qubits)
        C = res["tableau"]
        target_qubits = res["target_qubits"]
        circuit = res.get("circuit", None)
        
        print(f"Target qubits: {target_qubits}")
        if circuit:
            print(f"Circuit depth: {len(circuit)}")
        
        # Verify conjugation results
        print("\nConjugation verification:")
        for i, q in enumerate(target_qubits):
            S = parse_sparse_pauli(pairs[i][0], num_qubits=num_qubits)
            D = parse_sparse_pauli(pairs[i][1], num_qubits=num_qubits)
            out_S = C(S)
            out_D = C(D)

            print(f'{ps_to_sparse(S)} -> {ps_to_sparse(out_S)}, {ps_to_sparse(D)} -> {ps_to_sparse(out_D)} \n')
            want_S = stim.PauliString(num_qubits)
            want_S[q] = "Z"
            want_D = stim.PauliString(num_qubits)
            want_D[q] = "X"

            s_match = out_S == want_S
            d_match = out_D == want_D
            print(f"  Pair {i}: S {pairs[i][0]} -> {ps_to_sparse(out_S)} (target: Z{q}) ✓" if s_match else f"  Pair {i}: S {pairs[i][0]} -> {ps_to_sparse(out_S)} (target: Z{q}) ✗")
            print(f"  Pair {i}: D {pairs[i][1]} -> {ps_to_sparse(out_D)} (target: X{q}) ✓" if d_match else f"  Pair {i}: D {pairs[i][1]} -> {ps_to_sparse(out_D)} (target: X{q}) ✗")
        
        return res
    except Exception as e:
        print(f"ERROR: {e}")
        return None


## Example 1: Original 5-Qubit Test Case
The reference example from the test suite.

In [48]:
# Original test case from clifford_from_map tests
pairs_5q = [
("Z1 X3", "Z3"),
("Z0 Z2 X4", "X2"),
("Y0 Y1 Z3 Z4", "Z0"),
("Z0 X1 X2 Z3 Z4", "Z0 Z1"),
]

test_5q = test_clifford_synthesis(pairs_5q, num_qubits=5) 



=== Test ===
Input pairs (S, D): [('Z1 X3', 'Z3'), ('Z0 Z2 X4', 'X2'), ('Y0 Y1 Z3 Z4', 'Z0'), ('Z0 X1 X2 Z3 Z4', 'Z0 Z1')]
Number of qubits: 5
Target qubits: [0, 1, 2, 3]
Circuit depth: 20

Conjugation verification:
Z1 X3 -> Z0, Z3 -> X0 

  Pair 0: S Z1 X3 -> Z0 (target: Z0) ✓
  Pair 0: D Z3 -> X0 (target: X0) ✓
Z0 Z2 X4 -> Z1, X2 -> X1 

  Pair 1: S Z0 Z2 X4 -> Z1 (target: Z1) ✓
  Pair 1: D X2 -> X1 (target: X1) ✓
Y0 Y1 Z3 Z4 -> Z2, Z0 -> X2 

  Pair 2: S Y0 Y1 Z3 Z4 -> Z2 (target: Z2) ✓
  Pair 2: D Z0 -> X2 (target: X2) ✓
Z0 X1 X2 Z3 Z4 -> Z3, Z0 Z1 -> X3 

  Pair 3: S Z0 X1 X2 Z3 Z4 -> Z3 (target: Z3) ✓
  Pair 3: D Z0 Z1 -> X3 (target: X3) ✓


In [49]:
res = synthesize_clifford_from_sd_pairs(pairs_5q, num_qubits=5)
C = res["tableau"]
target_qubits = res["target_qubits"]
circuit = res.get("circuit", None)


## Example 2: Simple 3-Qubit Case
A minimal example with 3 qubits and simple stabilizer-destabilizer pairs.

In [59]:
# Simple 3-qubit case
pairs_3q = [
    ("Z0 Z1", "X0 Z1"),
    ("Z1 Z2", "Z1 X2"),
]

result_3q = test_clifford_synthesis(pairs_3q, num_qubits=3, description="Simple 3-Qubit Case")



=== Simple 3-Qubit Case ===
Input pairs (S, D): [('Z0 Z1', 'X0 Z1'), ('Z1 Z2', 'Z1 X2')]
Number of qubits: 3
ERROR: The given generator outputs don't describe a valid Clifford operation.
They don't preserve commutativity.
Everything must commute, except for X_k anticommuting with Z_k for each k.


## Example 3: Single Qubit Pairs
Testing with individual qubit stabilizers and destabilizers.

In [51]:
# Single qubit pairs
pairs_single = [
    ("Z0", "X0"),
    ("Z1", "X1"),
    ("Z2", "X2"),
]

result_single = test_clifford_synthesis(pairs_single, num_qubits=3, description="Single Qubit Pairs")



=== Single Qubit Pairs ===
Input pairs (S, D): [('Z0', 'X0'), ('Z1', 'X1'), ('Z2', 'X2')]
Number of qubits: 3
Target qubits: [0, 1, 2]
Circuit depth: 1

Conjugation verification:
Z0 -> Z0, X0 -> X0 

  Pair 0: S Z0 -> Z0 (target: Z0) ✓
  Pair 0: D X0 -> X0 (target: X0) ✓
Z1 -> Z1, X1 -> X1 

  Pair 1: S Z1 -> Z1 (target: Z1) ✓
  Pair 1: D X1 -> X1 (target: X1) ✓
Z2 -> Z2, X2 -> X2 

  Pair 2: S Z2 -> Z2 (target: Z2) ✓
  Pair 2: D X2 -> X2 (target: X2) ✓


## Example 4: Mixed Pauli Operators
Using Y operators and more complex combinations.

In [52]:
# Mixed Pauli operators with Y
pairs_mixed = [
    ("Y0", "Y1"),
    ("X1 Z2", "Z0 X2"),
    ("Z0 Y1 X2", "X0 Z1"),
]

result_mixed = test_clifford_synthesis(pairs_mixed, num_qubits=3, description="Mixed Pauli Operators")



=== Mixed Pauli Operators ===
Input pairs (S, D): [('Y0', 'Y1'), ('X1 Z2', 'Z0 X2'), ('Z0 Y1 X2', 'X0 Z1')]
Number of qubits: 3
ERROR: pair 0 invalid: S_i must anticommute with D_i


## Example 5: From Stabilizer Code Canonicalization
Generate stabilizer-destabilizer pairs from a stabilizer code and synthesize Clifford.

In [53]:
# Start with a set of stabilizers and canonicalize
stabilizers = ['X0 X1', 'X1 X2', 'Z0 Z1 Z2']
print(f"Input stabilizers: {stabilizers}")

# Canonicalize the stabilizer code
stabcan = canonicalize_stabilizer_code(stabilizers)
print(f"Canonicalized stabilizers: {stabcan['stabilizers_sparse']}")
print(f"Destabilizers: {stabcan['destabilizers_sparse']}")

# Create pairs and test synthesis
sd_pairs = list(zip(stabcan['stabilizers_sparse'], stabcan['destabilizers_sparse']))
result_canon = test_clifford_synthesis(sd_pairs, num_qubits=3, description="From Canonicalized Stabilizer Code")

# Show logical operators if found
if stabcan['logical_x_sparse'] and stabcan['logical_z_sparse']:
    print(f"Logical X operators: {stabcan['logical_x_sparse']}")
    print(f"Logical Z operators: {stabcan['logical_z_sparse']}")


Input stabilizers: ['X0 X1', 'X1 X2', 'Z0 Z1 Z2']
Canonicalized stabilizers: ['X0 X1', 'X1 X2', 'Z0 Z1 Z2']
Destabilizers: ['Z0', 'Z2', 'X1']

=== From Canonicalized Stabilizer Code ===
Input pairs (S, D): [('X0 X1', 'Z0'), ('X1 X2', 'Z2'), ('Z0 Z1 Z2', 'X1')]
Number of qubits: 3
Target qubits: [0, 1, 2]
Circuit depth: 5

Conjugation verification:
X0 X1 -> Z0, Z0 -> X0 

  Pair 0: S X0 X1 -> Z0 (target: Z0) ✓
  Pair 0: D Z0 -> X0 (target: X0) ✓
X1 X2 -> Z1, Z2 -> X1 

  Pair 1: S X1 X2 -> Z1 (target: Z1) ✓
  Pair 1: D Z2 -> X1 (target: X1) ✓
Z0 Z1 Z2 -> Z2, X1 -> X2 

  Pair 2: S Z0 Z1 Z2 -> Z2 (target: Z2) ✓
  Pair 2: D X1 -> X2 (target: X2) ✓


## Example 6: Larger 6-Qubit System
Testing scalability with a larger system.

In [54]:
# 6-qubit system
pairs_6q = [
    ("Z0 Z1", "X1 X2"),
    ("Z2 Z3", "X3 X4"),
    ("Z4 Z5", "X5 X0"),
    ("X0 X2 X4", "Z1 Z3 Z5"),
]

result_6q = test_clifford_synthesis(pairs_6q, num_qubits=6, description="6-Qubit System")



=== 6-Qubit System ===
Input pairs (S, D): [('Z0 Z1', 'X1 X2'), ('Z2 Z3', 'X3 X4'), ('Z4 Z5', 'X5 X0'), ('X0 X2 X4', 'Z1 Z3 Z5')]
Number of qubits: 6
ERROR: pair 3 invalid: S_i must anticommute with D_i


## Analysis and Verification Tools
Additional tools for analyzing the synthesized Cliffords.

In [55]:
def analyze_tableau(tableau, num_qubits, description="Tableau"):
    """Analyze properties of a Stim tableau."""
    print(f"\n=== {description} Analysis ===")
    print(f"Tableau size: {num_qubits} qubits")
    
    # Test some basic Pauli conjugations
    test_paulis = ["X0", "Z0", "Y0"]
    if num_qubits > 1:
        test_paulis.extend(["X0 X1", "Z0 Z1", "X0 Z1"])
    
    print("Sample conjugations:")
    for pauli_str in test_paulis[:min(6, len(test_paulis))]:
        try:
            p = parse_sparse_pauli(pauli_str, num_qubits=num_qubits)
            conjugated = tableau(p)
            print(f"  {pauli_str} -> {ps_to_sparse(conjugated)}")
        except:
            print(f"  {pauli_str} -> [error]")

# Analyze some of our results
if result_5q:
    analyze_tableau(result_5q["tableau"], 5, "5-Qubit Original")

if result_3q:
    analyze_tableau(result_3q["tableau"], 3, "3-Qubit Simple")

if result_canon:
    analyze_tableau(result_canon["tableau"], 3, "Canonicalized Code")



=== 5-Qubit Original Analysis ===
Tableau size: 5 qubits
Sample conjugations:
  X0 -> X1 Y2 Y3
  Z0 -> X2
  Y0 -> X1 Z2 Y3
  X0 X1 -> X0 X1 Z2 X3 Y4
  Z0 Z1 -> X3
  X0 Z1 -> X1 Z2 Z3

=== Canonicalized Code Analysis ===
Tableau size: 3 qubits
Sample conjugations:
  X0 -> Z0 X2
  Z0 -> X0
  Y0 -> Y0 X2
  X0 X1 -> Z0
  Z0 Z1 -> X1 Z2
  X0 Z1 -> Y0 X1 Y2


## Circuit Evolution Demo
Show how Pauli strings evolve through the synthesized circuits.

In [56]:
def evolve_pauli_via_circuit(pauli_string, circuit):
    """Evolve a Pauli string through a Stim circuit using tableau simulation."""
    if isinstance(pauli_string, str):
        pauli_string = parse_sparse_pauli(pauli_string, num_qubits=circuit.num_qubits)
    
    tableau = circuit.to_tableau()
    
    # Build result by conjugating each single-qubit Pauli
    result = stim.PauliString(circuit.num_qubits)
    for q in range(circuit.num_qubits):
        op = pauli_string[q]
        if op == 0:  # Identity
            continue
        elif op == 1:  # X
            term = tableau.x_output(q)
        elif op == 3:  # Z  
            term = tableau.z_output(q)
        elif op == 2:  # Y
            term = tableau.x_output(q) * tableau.z_output(q)
        else:
            raise ValueError(f"Unexpected Pauli opcode {op}")
        result *= term
    
    result *= pauli_string.sign
    return result

# Test circuit evolution for cases that have circuits
for name, result in [("5-Qubit", result_5q), ("3-Qubit", result_3q), ("Canonical", result_canon)]:
    if result and "circuit" in result:
        print(f"\n=== {name} Circuit Evolution ===")
        circuit = result["circuit"]
        print(f"Circuit has {len(circuit)} operations")
        
        # Test evolution of a few Pauli strings
        test_strings = ["X0", "Z0"]
        if circuit.num_qubits > 1:
            test_strings.append("X0 Z1")
        
        for pauli_str in test_strings:
            try:
                evolved = evolve_pauli_via_circuit(pauli_str, circuit)
                print(f"  {pauli_str} -> {ps_to_sparse(evolved)}")
            except Exception as e:
                print(f"  {pauli_str} -> [error: {e}]")



=== 5-Qubit Circuit Evolution ===
Circuit has 20 operations
  X0 -> X1 Y2 Y3
  Z0 -> X2
  X0 Z1 -> X1 Z2 Z3

=== Canonical Circuit Evolution ===
Circuit has 5 operations
  X0 -> Z0 X2
  Z0 -> X0
  X0 Z1 -> Y0 X1 Y2


## Commutation Testing
Verify commutation properties of the synthesized stabilizer-destabilizer pairs.

In [57]:
def verify_sd_commutation(pairs, description="Pairs"):
    """Verify that stabilizer-destabilizer pairs have correct commutation relations."""
    print(f"\n=== {description} Commutation Verification ===")
    
    stabilizers = [pair[0] for pair in pairs]
    destabilizers = [pair[1] for pair in pairs]
    
    print("Stabilizer-Stabilizer commutation (should all be True):")
    for i, s1 in enumerate(stabilizers):
        comm_results = commutes_sparse(s1, stabilizers)
        print(f"  {s1} with all: {comm_results}")
    
    print("\nDestabilizer-Destabilizer commutation (should all be True):")
    for i, d1 in enumerate(destabilizers):
        comm_results = commutes_sparse(d1, destabilizers)
        print(f"  {d1} with all: {comm_results}")
    
    print("\nStabilizer-Destabilizer commutation (diagonal should be False):")
    for i, s in enumerate(stabilizers):
        comm_results = commutes_sparse(s, destabilizers)
        print(f"  {s} with destabilizers: {comm_results} (position {i} should be False)")

# Test commutation for our examples
verify_sd_commutation(pairs_5q, "5-Qubit Original")
verify_sd_commutation(pairs_3q, "3-Qubit Simple")
if result_canon:
    verify_sd_commutation(sd_pairs, "Canonicalized Code")



=== 5-Qubit Original Commutation Verification ===
Stabilizer-Stabilizer commutation (should all be True):
  Z1 X3 with all: [True, True, True, True]
  Z0 Z2 X4 with all: [True, True, True, True]
  Y0 Y1 Z3 Z4 with all: [True, True, True, True]
  Z0 X1 X2 Z3 Z4 with all: [True, True, True, True]

Destabilizer-Destabilizer commutation (should all be True):
  Z3 with all: [True, True, True, True]
  X2 with all: [True, True, True, True]
  Z0 with all: [True, True, True, True]
  Z0 Z1 with all: [True, True, True, True]

Stabilizer-Destabilizer commutation (diagonal should be False):
  Z1 X3 with destabilizers: [False, True, True, True] (position 0 should be False)
  Z0 Z2 X4 with destabilizers: [True, False, True, True] (position 1 should be False)
  Y0 Y1 Z3 Z4 with destabilizers: [True, True, False, True] (position 2 should be False)
  Z0 X1 X2 Z3 Z4 with destabilizers: [True, True, True, False] (position 3 should be False)

=== 3-Qubit Simple Commutation Verification ===
Stabilizer-Stab

## Summary and Performance Notes
Overview of all test cases and their results.

In [58]:
# Summary of all test cases
print("=== SUMMARY OF ALL TEST CASES ===")
test_results = [
    ("5-Qubit Original", result_5q, pairs_5q),
    ("3-Qubit Simple", result_3q, pairs_3q),
    ("Single Qubit Pairs", result_single, pairs_single),
    ("Mixed Pauli Operators", result_mixed, pairs_mixed),
    ("Canonicalized Code", result_canon, sd_pairs if 'sd_pairs' in locals() else []),
    ("6-Qubit System", result_6q, pairs_6q),
]

successful_tests = 0
total_tests = len(test_results)

for name, result, pairs in test_results:
    status = "✓ SUCCESS" if result else "✗ FAILED"
    num_pairs = len(pairs)
    print(f"{name}: {status} ({num_pairs} pairs)")
    if result:
        successful_tests += 1
        target_qubits = result.get("target_qubits", [])
        print(f"  Target qubits: {target_qubits}")

print(f"\nOverall: {successful_tests}/{total_tests} tests successful")

if successful_tests == total_tests:
    print("All Clifford synthesis tests passed! The implementation is working correctly.")
else:
    print("Some tests failed. Check the error messages above for details.")


=== SUMMARY OF ALL TEST CASES ===
5-Qubit Original: ✓ SUCCESS (4 pairs)
  Target qubits: [0, 1, 2, 3]
3-Qubit Simple: ✗ FAILED (2 pairs)
Single Qubit Pairs: ✓ SUCCESS (3 pairs)
  Target qubits: [0, 1, 2]
Mixed Pauli Operators: ✗ FAILED (3 pairs)
Canonicalized Code: ✓ SUCCESS (3 pairs)
  Target qubits: [0, 1, 2]
6-Qubit System: ✗ FAILED (4 pairs)

Overall: 3/6 tests successful
Some tests failed. Check the error messages above for details.
